<a href="https://colab.research.google.com/github/shiwoei-ai/ai_tech_and_biz_app_course/blob/main/Workshop2b_r2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# 步驟 1: 環境設定與安裝、導入函式庫
# ============================================================

# 安裝 ucimlrepo 套件：從 UCI 資料庫下載資料
!pip install ucimlrepo

# 常用繪圖與資料分析工具函式庫
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo

# 建立模型所需功能
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# scikit-learn 工具
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    classification_report,
    confusion_matrix
)

In [ ]:
# ============================================================
# 步驟 2: 載入與檢視資料集
# ============================================================

# 從 UCI Repository 抓取 Bank Marketing 資料集 (ID=222)
bank_marketing = fetch_ucirepo(id=222)

# 分離特徵 (X) 與目標 (y)
X = bank_marketing.data.features
y = bank_marketing.data.targets

# 初步資料探索
print("特徵資料 (X) 的前 5 筆:")
print(X.head())
print("\n" + "=" * 50 + "\n")

print("目標變數 (y) 的分布情況:")
print(y.value_counts())


In [ ]:
# ============================================================
# 步驟 3: 資料前處理
# ============================================================

# 1. 識別不同類型的欄位
categorical_features = X.select_dtypes(
    include=['object']
).columns
numerical_features = X.select_dtypes(
    include=['number']
).columns

print(f"類別型特徵: {list(categorical_features)}")
print(f"數值型特徵: {list(numerical_features)}")
print("\n" + "=" * 50 + "\n")

# 2. 建立 ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'),
               categorical_features)
    ]
)

# 3. 處理目標變數
y = y.replace({'no': 0, 'yes': 1})

# 4. 切割訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 轉換資料
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"處理後的訓練資料維度: {X_train_processed.shape}")
print(f"處理後的測試資料維度: {X_test_processed.shape}")


In [ ]:
# ============================================================
# 步驟 4: 建立 DNN 模型
# ============================================================

# 可調整的模型超參數
LEARNING_RATE = 0.01
MOMENTUM = 0.9
DROPOUT_RATE = 0.5

# 建立 Sequential 模型
# Sequential 表示我們會按順序一層一層堆疊網路結構
model = Sequential()

# 開始堆疊神經網路層

# --- 第一層隱藏層 ---
# Dense(64)：建立一層全連接層，含有 64 個神經元
# activation='relu': 使用 ReLU 作為激活函數
# input_shape: 指定輸入維度，此處為處理後特徵數（欄位數）
# Dropout 可在訓練時隨機丟棄部分神經元，
# 目的在避免 overfitting，讓模型有更好的預測能力。
model.add(Dense(
    64,
    activation='relu',
    input_shape=(X_train_processed.shape[1],)
))
model.add(Dropout(DROPOUT_RATE))

# --- 第二層隱藏層 ---
# Dense(32)：第二層全連接層，神經元數量減半（32 個）
# 逐層縮小神經元數量是常見做法，讓網路逐漸壓縮資訊。
model.add(Dense(32, activation='relu'))
model.add(Dropout(DROPOUT_RATE))

# --- 輸出層 ---
# Dense(1)：單一輸出神經元，代表是否會申辦的機率
# activation='sigmoid' 會讓輸出值範圍 (0,1)，一般
# 用於輸出二元分類的機率。
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# ============================================================
# 步驟 5: 編譯 DNN 模型
# ============================================================

# 在 Keras/TensorFlow 中，編譯模型需要指定三要素：
# 1. optimizer（優化器）：決定模型用哪個優化器更新權重
# 2. loss（損失函數）：決定用哪個函數衡量模型的表現
# 3. metrics（評估指標）：決定訓練或測試過程中回報的表現數據

# 使用 SGD（stochastic gradient descent）優化器
# 此外我們設定 learning_rate（學習率），控制每次更新的步伐大小
# 另外，也加上 momentum（動量值），用來加速下降並避免震盪
optimizer = tf.keras.optimizers.SGD(
    learning_rate=LEARNING_RATE,
    momentum=MOMENTUM
)

# 編譯模型
# loss='binary_crossentropy' 適用於二元分類問題
# ，特別是輸出為 sigmoid 機率的情況。
# metrics=['accuracy'] 會在訓練與測試過程中回報
# accuracy（準確率）這個指標。
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 顯示模型結構
model.summary()


In [ ]:
# ============================================================
# 步驟 6: 訓練模型
# ============================================================

from tensorflow.keras.callbacks import EarlyStopping

# 可調整的超參數
EPOCHS = 50
BATCH_SIZE = 128

# 加入 EarlyStopping 機制
#
# monitor='val_loss' 會監控驗證集的損失是否改善。
# patience=10 表示如果連續 10 個 epoch 沒有改善，
# 就提前停止訓練。
# restore_best_weights=True 表示停止訓練後，自動
# 回復到最佳表現（即 val_loss 最小）時的權重。
# 加入 EarlyStopping 的好處在避免過度擬合（overfitting），
# 也可節省模型訓練時間。
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# 開始訓練
history = model.fit(
    X_train_processed,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test_processed, y_test),
    callbacks=[early_stopping],
    verbose=1
)


In [ ]:
# ============================================================
# 步驟 7: 評估最終模型與視覺化混淆矩陣
# ============================================================

import seaborn as sns

# 1. 在測試集上評估
print("在測試集上評估模型...")
test_loss, test_acc = model.evaluate(
    X_test_processed,
    y_test,
    verbose=0
)
print(f'\nTest Accuracy: {test_acc:.4f}')
print("\n" + "=" * 50 + "\n")

# 2. 產生預測與分類報告
y_pred_proba = model.predict(X_test_processed)
y_pred = (y_pred_proba > 0.4).astype(int)

print("分類報告 (Classification Report):")
print(classification_report(
    y_test,
    y_pred,
    target_names=['No (0)', 'Yes (1)']
))
print("\n" + "=" * 50 + "\n")

# 3. 繪製混淆矩陣
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['Predicted No (0)', 'Predicted Yes (1)'],
    yticklabels=['Actual No (0)', 'Actual Yes (1)']
)
plt.title('Confusion Matrix')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()


In [ ]:
# ============================================================
# 步驟 8: 互動式預測應用
# ============================================================

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# 1. 建立互動元件
index_dropdown = widgets.Dropdown(
    options=list(range(len(X_test))),
    value=1,
    description='選擇顧客:',
    layout={'width': '50%'},
    style={'description_width': 'initial'}
)

title_html = widgets.HTML(
    "<h2>銀行客戶申辦意願 - 互動預測</h2>"
)
output_area = widgets.Output()

# 2. 定義核心的預測與顯示函式
def on_dropdown_change(change):
    """當 Dropdown 的值改變時，觸發此函式"""
    customer_index = change['new']  # 新選擇的顧客編號

    with output_area:
        clear_output(wait=True)

        # A. 提取資料
        sample_original = X_test.iloc[customer_index].copy()
        sample_processed = X_test_processed[customer_index].reshape(1, -1)
        actual_label = y_test.iloc[customer_index].iloc[0]

        # B. 進行預測
        prediction_probability = model.predict(
            sample_processed,
            verbose=0
        )[0][0]
        predicted_label = 1 if prediction_probability > 0.4 else 0

        # C. 產生美化後的 HTML 輸出

        # 【中文翻譯】
        column_translation = {
            'age': '年齡',
            'job': '職業',
            'marital': '婚姻狀況',
            'education': '教育程度',
            'default': '信用違約',
            'balance': '帳戶餘額',
            'housing': '房貸',
            'loan': '個人信貸',
            'contact': '聯繫方式',
            'month': '月份',
            'day_of_week': '星期',
            'duration': '通話時長(秒)',
            'campaign': '本次聯繫次數',
            'pdays': '上次聯繫間隔(天)',
            'previous': '先前聯繫次數',
            'poutcome': '上次行銷結果',
            'y': '真實申辦情況'
        }

        # 將真實申辦情況(y)加入要顯示的資訊中
        sample_original['y'] = 'yes' if actual_label == 1 else 'no'

        info_df = sample_original.to_frame().T
        new_columns = [
            f"{col} ({column_translation.get(col, '')})"
            for col in info_df.columns
        ]
        info_df.columns = new_columns
        info_html = info_df.to_html(index=False)

        # 產生預測結果的 HTML
        pred_html = f"""
        <div style='line-height: 1.6;'>
            模型預測此顧客申辦的機率為:
            <b>{prediction_probability:.2%}</b><br>
            模型最終判斷:
            <b>{'會申辦 (1)' if predicted_label == 1 else '不會申辦 (0)'}</b><br>
            <hr>
            顧客真實情況:
            <b>{'會申辦 (1)' if actual_label == 1 else '不會申辦 (0)'}</b>
        </div>
        """

        # 產生對比結果的 HTML
        if predicted_label == actual_label:
            result_html = (
                "<div style='color: green; font-size: 20px;'>"
                "<b>✅ 模型預測正確！</b></div>"
            )
        else:
            result_html = (
                "<div style='color: red; font-size: 20px;'>"
                "<b>❌ 模型預測錯誤。</b></div>"
            )

        # D. 顯示所有內容
        display(HTML(f"<h4>正在檢視 測試集中的顧客 #{customer_index}</h4>"))
        display(HTML("<h4>--- 顧客原始資訊 ---</h4>"))
        display(HTML(info_html))
        display(HTML("<h4>--- 模型預測分析 ---</h4>"))
        display(HTML(pred_html))
        display(HTML("<h4>--- 對比結果 ---</h4>"))
        display(HTML(result_html))


# 3. 綁定事件與建立最終介面
index_dropdown.observe(on_dropdown_change, names='value')
app = widgets.VBox([
    title_html,
    index_dropdown,
    output_area
])

# 4. 顯示應用程式，並觸發初始顯示
display(app)
on_dropdown_change({'new': index_dropdown.value})